<a href="https://colab.research.google.com/github/shanikairoshi/Communication-Efficient-DUQFL/blob/main/main_seededCom_genome_0.12_bestAll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the latest implementation
This includes;
1. agg
2. clasical and quantum communication
3. teleportaion options

flowchart LR
  %% ----------- Inputs / Config -----------
  A[Start Round t\nConfig:\n• use_deep_unfolding (DU)\n• aggregation ∈ {best, fedavg}\n• uplink_mode ∈ {classical_full, seeded_sparse, quantum}\n• downlink_mode ∈ {classical_full, seeded, quantum}\n• gamma (warm-start)\n• teleport noise, shots]:::cfg

  %% ----------- Broadcast -----------
  A --> B[Server has Global Params w^{t}\n(also prev_broadcast_params = w^{t})]
  B --> C{Downlink mode?}

  C -->|classical_full| C1[[Broadcast full w^{t}]]
  C -->|seeded (seeded mask + scale)| C2[[Send seed & mask\nClients reconstruct ŵ^{t}]]
  C -->|quantum (teleportation)| C3[[Teleport w^{t}\n(θ → θ′ via Bell pair)\nRecord fidelity, β shrink]]

  C1 --> D
  C2 --> D
  C3 --> D

  %% ----------- Local Training -----------
  subgraph S[Each Client i]
    direction TB
    D[Client i receives parameters\nw_i^{t,0} ← w^{t} (reconstructed if seeded/quantum)] --> E[Warm-start LR,PERT:\nLR_i^0 = (1-γ)LR_init + γLR_i^{t-1}\nPERT_i^0 = (1-γ)PERT_init + γPERT_i^{t-1}]
    E --> F{use_deep_unfolding?}
    F -->|Yes| G[Repeat k steps:\nfit → meta-update {LR,PERT}\nclip to trust region\nset w_i^{t,k}]
    F -->|No| H[Single fit with fixed LR, PERT\nset w_i^{t,1}]
    G --> I[Compute metrics (train/test acc,\nclient size, objective)]
    H --> I
    I --> J{Uplink mode?}
    J -->|classical_full| J1[[Send full weights w_i^{t,*}]]
    J -->|seeded_sparse| J2[[Send seeded sparse Δ_i\n(quantized top-k wrt anchor)]]
    J -->|quantum| J3[[Teleport Δ_i or w_i^{t,*}\n(log fidelity)]]
  end

  %% ----------- Aggregation -----------
  J1 --> K
  J2 --> K
  J3 --> K

  K{Aggregation?} -->|best\n(validation-gated τ-mix)| K1[[Pick client j = argmin L_val\nIf improve ≥ ε: w^{t+1} ← (1-τ)w^{t} + τ w_j]]
  K -->|fedavg| K2[[Size-weighted average\nw^{t+1} ← Σ (n_i/N) w_i^{t,*}]]

  K1 --> L[Log round metrics:\nacc_global, L_val,\nbytes_up/down, latency,\nfidelity stats, β]
  K2 --> L
  L --> M[Proceed to Round t+1]
  
  classDef cfg fill:#f5f7ff,stroke:#6b7cff,color:#222,stroke-width:1.2px;


In [1]:
# %%capture
!pip install genomic-benchmarks
!pip install qiskit qiskit_machine_learning qiskit_algorithms qiskit-aer




In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
from pathlib import Path
PROJ = Path.cwd() / "tduQFL_Project_QFLvsDUQFL_AggChoice_CommEff_v2"
if str(PROJ) not in sys.path:
    sys.path.insert(0, str(PROJ))
import sys
sys.path.append('/content/drive/MyDrive/Teleportation/tduQFL_Project_QFLvsDUQFL_AggChoice_CommEff_v2/tDuQFL_Project')
# ─── 5. Assemble filenames for each artifact ─────────────────────────────────
drive_root = "/content/drive/MyDrive/Teleportation/tduQFL_Project_QFLvsDUQFL_AggChoice_CommEff_v2/tDuQFL_Project/"

In [4]:

#from configs.dataset_genome_noniid.py import *     # swap to other configs as needed

In [5]:
from common.imports import *
from configs.dataset_genome import *     # swap to other configs as needed
from io_utils.naming import stamp_now, build_param_str, make_filenames

# main.py or notebook cell
from configs.base_config import *
from io_utils.naming import build_param_str, stamp_now, make_filenames


stamp, date_str = stamp_now()

best_client_csv_file, global_csv_file, local_csv_file, validation_csv_file = make_filenames(
    drive_root=drive_root,
    dataset_name=dataset_name,
    split_type=split_type,
    date_str=date_str,
    aggregation=aggregation,
    select_upload=select_upload,
    use_deep_unfolding=use_deep_unfolding,
    gamma=gamma,
    uplink_mode=uplink_mode,
    downlink_mode=downlink_mode,
    up_k_ratio=up_k_ratio,
    up_bits=up_bits,
    down_mask_ratio=down_mask_ratio,
    down_scale=down_scale,
    noise_preset=noise_preset,
    shots_used=shots_used,

)
print(best_client_csv_file)
print(global_csv_file)
print(local_csv_file)
print(validation_csv_file)

from io_utils.csv_logger import init_local_csv, init_best_csv, init_validation_csv

# Create folders + write headers
init_best_csv(best_client_csv_file)

local_headers = [
    "Federated Round", "Client Number", "Iteration",
    "Objective Function Value", "Training Accuracy", "Test Accuracy",
    "Learning Rate", "Perturbation"
]
init_local_csv(local_csv_file, local_headers)

init_validation_csv(validation_csv_file)

# Do NOT pre-init global_csv_file here because your save_accuracies_to_csv()
# already writes the header each time it runs (in 'w' mode).

/usr/local/lib/python3.12/dist-packages/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Qiskit: 1.4.5
qiskit_aer available?: True
/content/drive/MyDrive/Teleportation/tduQFL_Project_QFLvsDUQFL_AggChoice_CommEff_v2/tDuQFL_Project/Results/seededCommunication/Genome/bestall/tDuQFL_Genome_NONIID_AGG-best_SU-all_DU-on_g0p5_UL-seeded_sparse__DL-seeded__upk1pct__b8__dm10pct__ds0p001_Q-off_17_10_2025_Best_Client.csv
/content/drive/MyDrive/Teleportation/tduQFL_Project_QFLvsDUQFL_AggChoice_CommEff_v2/tDuQFL_Project/Results/seededCommunication/Genome/bestall/tDuQFL_Genome_NONIID_AGG-best_SU-all_DU-on_g0p5_UL-seeded_sparse__DL-seeded__upk1pct__b8__dm10pct__ds0p001_Q-off_17_10_2025_Global.csv
/content/drive/MyDrive/Teleportation/tduQFL_Project_QFLvsDUQFL_AggChoice_CommEff_v2/tDuQFL_Project/Results/seededCommunication/Genome/bestall/tDuQFL_Genome_NONIID_AGG-best_SU-all_DU-on_g0p5_UL-seeded_sparse__DL-

Load and Split data

run federated loop and plot

In [6]:
from training.loop import run_federated_training
from configs.base_config import use_teleportation as CFG_TEL, noise_preset, shots_used
from training.metrics import metrics_init, metrics_log_round, metrics_finalize, compute_auc,metrics_summarize
from viz.plots import plot_accuracy_curve, plot_val_loss, plot_time_per_round, plot_fidelity_vs_delta_acc, plot_beta_hist, plot_client_fairness_last_round
# Initialize metrics store once
metrics_store = metrics_init(
    log_path=os.path.join(drive_root, "teleport_metrics_Perturb_shrink.csv")
)

#new
from ml import optimizers as mlopt
from configs.base_config import drive_root
import os

mlopt.meta_trace_enable(
    path=os.path.join(drive_root, "meta_trace.csv"),  # or None to skip CSV
    every=5                                           # print every 5 callbacks
)

# examples/run_switch.py
from configs.base_config import *
from training.data_factory import build_clients_and_meta
from training.loop import run_federated_training
from training.metrics import metrics_init
import os, numpy as np

param_str = build_param_str(
    num_clients,
    num_federated_layers,
    num_deep_unfolding_iterations,
    initial_learning_rate,
    initial_perturbation,
)


# 1) Build clients + meta from config
clients, test_sequences, test_labels, num_features = build_clients_and_meta(
    dataset_name=dataset_name,
    split_type=split_type,
    num_clients=num_clients,
    num_epochs=num_epochs,
    samples_per_epoch=samples_per_epoch,
    word_size=word_size,
    global_seed=global_seed,
    mnist_n_features=mnist_n_features,
    mnist_digit_a=mnist_digit_a,
    mnist_digit_b=mnist_digit_b,
    breast_pca_n_features=breast_pca_n_features,
    non_iid_ratio=non_iid_ratio,
    quantity_variation=quantity_variation,
    noniid_seed=noniid_seed,
)

# 2) Validation set (use test as val if you don’t have a separate split)
X_val, y_val = test_sequences, test_labels

# 3) Outputs
os.makedirs(drive_root, exist_ok=True)


metrics = metrics_init(log_path=os.path.join(drive_root, "round_metrics.csv"))

# 4) Train
global_acc, clients_train, clients_test, round_times, val_losses, info_last = run_federated_training(
    clients=clients,
    num_federated_layers=num_federated_layers,
    num_deep_unfolding_iterations=num_deep_unfolding_iterations,
    initial_learning_rate=initial_learning_rate,
    initial_perturbation=initial_perturbation,
    num_features=num_features,
    best_client_csv_file=best_client_csv_file,
    global_csv_file=global_csv_file,
    local_csv_file=local_csv_file,
    validation_csv_file=validation_csv_file,
    test_sequences=test_sequences, test_labels=test_labels,
    X_val=X_val,
    y_val=y_val,
    metrics=metrics,
    aggregation=aggregation,                  # "best" or "fedavg"
    select_upload=select_upload,              # "all" or "winner_only" (used for best)
    use_deep_unfolding=use_deep_unfolding,    # True/False
    uplink_mode=uplink_mode,                  # "classical_full" | "seeded_sparse" | "quantum"
    downlink_mode=downlink_mode,              # "classical_full" | "seeded" | "quantum"
    down_seed_base=down_seed_base,
    down_scale=down_scale,
    down_mask_ratio=down_mask_ratio,
    up_bits=up_bits,
    up_k_ratio=up_k_ratio,
    use_teleportation_backend=True,
    noise_preset=noise_preset,
    shots_used=shots_used,
    client_hparams_csv_file=os.path.join(drive_root, f"hparams_{dataset_name}_{split_type}_{param_str}.csv"),
)

print("Global accuracy per round:", global_acc)

Training Progress:   0%|          | 0/10 [00:00<?, ?it/s]

[Round 0] Agg=best | Uplink=seeded_sparse | Downlink=seeded
[meta] cb#005 loss=0.486502 step=0.1005 g_raw=+0.047 g_sm=+0.003 acc=1 | LR→0.120240 PERT→0.120000 (scale=0.04)
[meta] cb#010 loss=0.476289 step=0.09739 g_raw=+0.042 g_sm=+0.009 acc=1 | LR→0.120481 PERT→0.120000 (scale=0.04)
[meta] cb#015 loss=0.462406 step=0.14 g_raw=+0.063 g_sm=+0.014 acc=1 | LR→0.120723 PERT→0.120000 (scale=0.04)
[meta] cb#020 loss=0.449768 step=0.04833 g_raw=+0.024 g_sm=+0.018 acc=1 | LR→0.120965 PERT→0.120001 (scale=0.04)
[meta] cb#025 loss=0.426743 step=0.1407 g_raw=+0.060 g_sm=+0.024 acc=1 | LR→0.121207 PERT→0.120001 (scale=0.04)
[client 0 | unfold 1] LR 0.120000->0.121207, PERT 0.120000->0.120001 | g_raw_mean=+0.029, g_sm_mean=+0.011
[meta] cb#030 loss=0.394083 step=0.0229 g_raw=+0.006 g_sm=+0.027 acc=1 | LR→0.121451 PERT→0.120002 (scale=0.04)
[meta] cb#035 loss=0.367569 step=0.1366 g_raw=+0.053 g_sm=+0.030 acc=1 | LR→0.121694 PERT→0.120003 (scale=0.04)
[meta] cb#040 loss=0.366905 step=0.02084 g_raw=+0

Training Progress:  10%|█         | 1/10 [16:39<2:29:55, 999.50s/it]

[Round   0] acc_g=0.614 (μ=0.540, σ=0.046, FG=0.097) | t=946.775s, val=0.599 | TEL=FALSE
[Round 0] ↓=24B ↑=110B | (Σ↓=24B, Σ↑=110B)
[Round 1] Agg=best | Uplink=seeded_sparse | Downlink=seeded
[meta] cb#005 loss=0.481281 step=0.01095 g_raw=+0.006 g_sm=+0.004 acc=1 | LR→0.123328 PERT→0.120005 (scale=0.04)
[meta] cb#010 loss=0.464358 step=0.1747 g_raw=+0.067 g_sm=+0.010 acc=1 | LR→0.123575 PERT→0.120005 (scale=0.04)
[meta] cb#015 loss=0.452170 step=0.06153 g_raw=+0.024 g_sm=+0.015 acc=1 | LR→0.123823 PERT→0.120006 (scale=0.04)
[meta] cb#020 loss=0.435924 step=0.003647 g_raw=+0.003 g_sm=+0.017 acc=1 | LR→0.124071 PERT→0.120006 (scale=0.04)
[meta] cb#025 loss=0.423922 step=0.01103 g_raw=+0.005 g_sm=+0.019 acc=1 | LR→0.124320 PERT→0.120007 (scale=0.04)
[client 0 | unfold 1] LR 0.123082->0.124320, PERT 0.120005->0.120007 | g_raw_mean=+0.025, g_sm_mean=+0.011
[meta] cb#030 loss=0.414504 step=0.06018 g_raw=+0.028 g_sm=+0.019 acc=1 | LR→0.124569 PERT→0.120007 (scale=0.04)
[meta] cb#035 loss=0.40

Training Progress:  20%|██        | 2/10 [33:05<2:12:10, 991.36s/it]

...

[Round   7] acc_g=0.721 (μ=0.497, σ=0.013, FG=0.026) | t=776.251s, val=0.617 | TEL=FALSE
[Round 7] ↓=24B ↑=110B | (Σ↓=192B, Σ↑=880B)
[Round 8] Agg=best | Uplink=seeded_sparse | Downlink=seeded
[meta] cb#005 loss=0.504375 step=0.03829 g_raw=+0.014 g_sm=+0.004 acc=1 | LR→0.126713 PERT→0.120010 (scale=0.04)
[meta] cb#010 loss=0.488707 step=0.02375 g_raw=+0.010 g_sm=+0.010 acc=1 | LR→0.126966 PERT→0.120011 (scale=0.04)
[meta] cb#015 loss=0.462906 step=0.01726 g_raw=+0.007 g_sm=+0.016 acc=1 | LR→0.127221 PERT→0.120011 (scale=0.04)
[meta] cb#020 loss=0.443372 step=0.01122 g_raw=+0.002 g_sm=+0.020 acc=1 | LR→0.127476 PERT→0.120011 (scale=0.04)
[meta] cb#025 loss=0.420286 step=0.03148 g_raw=+0.009 g_sm=+0.023 acc=1 | LR→0.127732 PERT→0.120012 (scale=0.04)
[client 0 | unfold 1] LR 0.126459->0.127732, PERT 0.120010->0.120012 | g_raw_mean=+0.031, g_sm_mean=+0.013
[meta] cb#030 loss=0.409512 step=0.06293 g_raw=+0.024 g_sm=+0.023 acc=1 | LR→0.127988 PERT→0.120013 (scale=0.04)
[meta] cb#035 loss=0.3

Training Progress:  90%|█████████ | 9/10 [2:21:08<14:47, 887.42s/it]

[Round   8] acc_g=0.802 (μ=0.501, σ=0.006, FG=0.014) | t=821.053s, val=0.615 | TEL=FALSE
[Round 8] ↓=24B ↑=110B | (Σ↓=216B, Σ↑=990B)
[Round 9] Agg=best | Uplink=seeded_sparse | Downlink=seeded
[meta] cb#005 loss=0.520354 step=0.01545 g_raw=+0.005 g_sm=+0.003 acc=1 | LR→0.126731 PERT→0.120011 (scale=0.04)
[meta] cb#010 loss=0.490070 step=0.03764 g_raw=+0.014 g_sm=+0.011 acc=1 | LR→0.126985 PERT→0.120011 (scale=0.04)
[meta] cb#015 loss=0.477456 step=0.1467 g_raw=+0.059 g_sm=+0.015 acc=1 | LR→0.127240 PERT→0.120012 (scale=0.04)
[meta] cb#020 loss=0.449883 step=0.08523 g_raw=+0.032 g_sm=+0.021 acc=1 | LR→0.127495 PERT→0.120012 (scale=0.04)
[meta] cb#025 loss=0.427822 step=0.03969 g_raw=+0.016 g_sm=+0.024 acc=1 | LR→0.127751 PERT→0.120013 (scale=0.04)
[client 0 | unfold 1] LR 0.126478->0.127751, PERT 0.120011->0.120013 | g_raw_mean=+0.032, g_sm_mean=+0.013
[meta] cb#030 loss=0.410008 step=0.09982 g_raw=+0.040 g_sm=+0.027 acc=1 | LR→0.128007 PERT→0.120013 (scale=0.04)
[meta] cb#035 loss=0.37

Training Progress: 100%|██████████| 10/10 [2:33:11<00:00, 919.20s/it]

[Round   9] acc_g=0.760 (μ=0.510, σ=0.013, FG=0.027) | t=672.649s, val=0.614 | TEL=FALSE
[Round 9] ↓=24B ↑=110B | (Σ↓=240B, Σ↑=1100B)
Global accuracy per round: [0.614, 0.639, 0.624, 0.637, 0.691, 0.602, 0.688, 0.721, 0.802, 0.76]
